<a href="https://colab.research.google.com/github/abzb1/UoS_AI_20026/blob/main/02_Fine-Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install uv
!uv pip install transformers accelerate peft trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 77.5 MB/s eta 0:00:00
Using Python 3.12.11 environment at: /usr
Resolved 62 packages in 630ms
Prepared 1 package in 36ms
Installed 1 package in 7ms
 + trl==0.21.0


In [2]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

In [3]:
# Load model and tokenizer
model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)

print(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 960, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=960, out_features=960, bias=False)
          (k_proj): Linear(in_features=960, out_features=320, bias=False)
          (v_proj): Linear(in_features=960, out_features=320, bias=False)
          (o_proj): Linear(in_features=960, out_features=960, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=960, out_features=2560, bias=False)
          (up_proj): Linear(in_features=960, out_features=2560, bias=False)
          (down_proj): Linear(in_features=2560, out_features=960, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((960,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((960,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((960,), eps=1e-05)
    (rotary_emb)

In [4]:
# test sample

messages = [
    {"role": "user", "content": "서울시립대 국제처 전화번호 알려줘."},
]
chat_text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

with torch.inference_mode():
    gen = model.generate(
        **inputs,
        max_new_tokens=128,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(gen[0], skip_special_tokens=False))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
서울시립대 국제처 전화번호 알려줘.<|im_end|>
<|im_start|>assistant
서울시립대 국제처 전화번호 알려줘.<|im_end|>


In [5]:
# prepare sample data

samples = [
    {
        "prompt": [{"role": "user", "content": "서울시립대 대표 전화번호 알려줘."}],
        "completion": [{"role": "assistant", "content": "서울시립대학교의 대표 전화번호는 02-6490-6114 입니다."}]
    },
    {
        "prompt": [{"role": "user", "content": "서울시립대 국제처 전화번호 알려줘."}],
        "completion": [{"role": "assistant", "content": "서울시립대학교 국제처의 전화번호는 02-6490-6662 입니다."}]
    },
    {
        "prompt": [{"role": "user", "content": "서울시립대 전산정보원 전화번호 알려줘."}],
        "completion": [{"role": "assistant", "content": "서울시립대학교 전산정보원의 전화번호는 02-6490-6119 입니다."}]
    },
    {
        "prompt": [{"role": "user", "content": "서울시립대 재무과 전화번호 알려줘."}],
        "completion": [{"role": "assistant", "content": "서울시립대학교 재무과의 전화번호는 02-6490-6462 입니다."}]
    },
    {
        "prompt": [{"role": "user", "content": "서울시립대 총무과 전화번호 알려줘."}],
        "completion": [{"role": "assistant", "content": "서울시립대학교 총무과의 전화번호는 02-6490-6411 입니다."}]
    },
    {
        "prompt": [{"role": "user", "content": "서울시립대 학생과 전화번호 알려줘."}],
        "completion": [{"role": "assistant", "content": "서울시립대학교 학생과의 전화번호는 02-6490-6211 입니다."}]
    },
    {
        "prompt": [{"role": "user", "content": "서울시립대 인재개발실 전화번호 알려줘."}],
        "completion": [{"role": "assistant", "content": "서울시립대학교 인재개발실의 전화번호는 02-6490-6256 입니다."}]
    },
    {
        "prompt": [{"role": "user", "content": "서울시립대 산학협력단 전화번호 알려줘."}],
        "completion": [{"role": "assistant", "content": "서울시립대학교 산학협력단의 전화번호는 02-6490-6360 입니다."}]
    },
]
train_ds = Dataset.from_list(samples)
print(train_ds[0])

{'prompt': [{'content': '서울시립대 대표 전화번호 알려줘.', 'role': 'user'}], 'completion': [{'content': '서울시립대학교의 대표 전화번호는 02-6490-6114 입니다.', 'role': 'assistant'}]}


In [6]:
# prepare lora and trainer config

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules="all-linear",
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = SFTConfig(
    output_dir="./smollm-lora-sft",
    num_train_epochs=100,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=10,
    save_steps=10,
    bf16=True,
    gradient_checkpointing=False,
    completion_only_loss=True,
    packing=False,
    model_init_kwargs={"torch_dtype": torch.bfloat16},
    report_to="none",
)

In [7]:
# build trainer and train

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_ds,
    args=training_args,
    peft_config=peft_config,
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:389: UserWarning: You passed model_init_kwargs to the `SFTConfig`, but your model is already instantiated. The `model_init_kwargs` will be ignored.
  warnings.warn(


Step,Training Loss
10,0.661500
20,0.338400
30,0.139100
40,0.072900
50,0.056100
60,0.041300
70,0.020100
80,0.007900
90,0.005400
100,0.004900


TrainOutput(global_step=100, training_loss=0.13475962288677693, metrics={'train_runtime': 117.4916, 'train_samples_per_second': 6.809, 'train_steps_per_second': 0.851, 'total_flos': 240381037056000.0, 'train_loss': 0.13475962288677693})

In [8]:
# merge model and save

merged = trainer.model.merge_and_unload()
merged.save_pretrained("./smollm-lora-sft-merged")
tokenizer.save_pretrained("./smollm-lora-sft-merged")

('./smollm-lora-sft-merged/tokenizer_config.json',
 './smollm-lora-sft-merged/special_tokens_map.json',
 './smollm-lora-sft-merged/chat_template.jinja',
 './smollm-lora-sft-merged/vocab.json',
 './smollm-lora-sft-merged/merges.txt',
 './smollm-lora-sft-merged/added_tokens.json',
 './smollm-lora-sft-merged/tokenizer.json')

In [9]:
# test after train

merged.eval()
device = next(merged.parameters()).device

messages = [
    {"role": "user", "content": "서울시립대 국제처 전화번호 알려줘."},
]

chat_text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
inputs = tokenizer(chat_text, return_tensors="pt").to(device)

with torch.inference_mode():
    gen = merged.generate(
        **inputs,
        max_new_tokens=128,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(gen[0], skip_special_tokens=False))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
서울시립대 국제처 전화번호 알려줘.<|im_end|>
<|im_start|>assistant
서울시립대학교 국제처의 전화번호는 02-6490-6662 입니다.<|im_end|>
